## run this cell before starting spark cluster connection (corrects module error)

In [26]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [27]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [28]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [29]:
df_lookup = spark.read.csv("/user/sprenner/lookup.csv", sep="\t")

In [30]:
df_lookup.count()

827642445

In [31]:
#df_lookup.limit(10).toPandas()

In [32]:
df_lookup_renamed = df_lookup.toDF('SCOPE', 'NAME', 'CHILD_SCOPE', 'CHILD_NAME', 'DID_TYPE', 'CHILD_TYPE', 
                                   'LENGTH', 'BYTES', 'ADLER32', 'MD5', 'RULE_EVALUATION', 'UPDATED_AT', 
                                   'CREATED_AT', 'EVENTS')

In [33]:
# egal ob hourly oder nicht, die lookup information bleibt gleich
df_april = spark.read.json("/user/sprenner/training_official_april_cleaned_datasets.json")

In [34]:
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["datasetScope"], df_april["dataset"]))
df_april.createOrReplaceTempView("april")
df_lookup_renamed = df_lookup_renamed.withColumn("join_key_lookup", concat_ws(':', df_lookup_renamed["SCOPE"], df_lookup_renamed["NAME"]))
df_lookup_renamed.createOrReplaceTempView("lookup_renamed")

In [35]:
spark.sql("SELECT * FROM lookup_renamed WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [SCOPE, NAME, CHILD_SCOPE, CHILD_NAME, DID_TYPE, CHILD_TYPE, LENGTH, BYTES, ADLER32, MD5, RULE_EVALUATION, UPDATED_AT, CREATED_AT, EVENTS, join_key_lookup]
Index: []

In [36]:
df_lookup_reduced = spark.sql("SELECT *\
                                 FROM lookup_renamed\
                                 WHERE SCOPE != 'panda' AND DID_TYPE == 'D' AND (NAME LIKE 'data%' OR NAME LIKE 'mc%')")
df_lookup_reduced.count()

625826659

In [37]:
df_april.limit(5).toPandas()

account  accountHash clientState  clientStateHash  \
0  hegormse -2036406190   DONE       -878527175         
1  vkitali   1625481889   DONE       -878527175         
2  mproffit  1290301367   DONE       -878527175         
3  mproffit  1290301367   DONE       -878527175         
4  mproffit  1290301367   DONE       -878527175         

                                                                                 dataset  \
0  data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
1  data15_13TeV.00284285.physics_Main.deriv.DAOD_EXOT5.r9264_p3083_p3481_tid13613132_00    
2  data15_13TeV.00267073.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466289_00    
3  data15_13TeV.00267162.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466294_00    
4  data15_13TeV.00276181.physics_Main.deriv.DAOD_EXOT8.r9412_p3083_p3481_tid13466382_00    

   datasetHash  datasetScope  datasetScopeHash  \
0  1268366167   data15_13TeV  1217185487         
1 -106274841    data15_13TeV  1217185487         
2 -1055546401   data15_13TeV  1217185487         
3 -1582503124   data15_13TeV  1217185487         
4 -1817287115   data15_13TeV  1217185487         

                    dataset_version  dataset_versionHash     datatype  \
0  r9264_p3083_p3402_tid12752857_00  1724430760           DAOD_EXOT22   
1  r9264_p3083_p3481_tid13613132_00 -1787190451           DAOD_EXOT5    
2  r9264_p3083_p3481_tid13466289_00  772360213            DAOD_EXOT8    
3  r9264_p3083_p3481_tid13466294_00  2105334052           DAOD_EXOT8    
4  r9412_p3083_p3481_tid13466382_00 -959087507            DAOD_EXOT8    

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0 -141841045     download  59378080       1.14.11     -2044360925         
1  1547296573    download  59378080       1.14.11     -2044360925         
2 -754389316     download  59378080       1.14.11     -2044360925         
3 -754389316     download  59378080       1.14.11     -2044360925         
4 -754389316     download  59378080       1.14.11     -2044360925         

      fileScope                                  filename  filenameHash  \
0  data15_13TeV  DAOD_EXOT22.12752857._000007.pool.root.1 -1554238082     
1  data15_13TeV  DAOD_EXOT5.13613132._000112.pool.root.1  -2031820614     
2  data15_13TeV  DAOD_EXOT8.13466289._000043.pool.root.1  -286165056      
3  data15_13TeV  DAOD_EXOT8.13466294._000153.pool.root.1  -1251159597     
4  data15_13TeV  DAOD_EXOT8.13466382._000031.pool.root.1   1916844419     

   filesize_double  hits               hostname  hostnameHash prod_step  \
0  6.236237e+08     1     lxplus104.cern.ch      494806480     deriv      
1  2.173912e+09     1     nafhh-atlas03.desy.de -492212293     deriv      
2  1.038860e+07     1     lxplus065.cern.ch      922885429     deriv      
3  1.381898e+07     1     lxplus065.cern.ch      922885429     deriv      
4  2.010680e+07     1     lxplus061.cern.ch     -1008250248    deriv      

   prod_stepHash       project  projectHash protocol  protocolHash  \
0 -1058503065     data15_13TeV  1217185487   srm      863299160      
1 -1058503065     data15_13TeV  1217185487   srm      863299160      
2 -1058503065     data15_13TeV  1217185487   srm      863299160      
3 -1058503065     data15_13TeV  1217185487   srm      863299160      
4 -1058503065     data15_13TeV  1217185487   srm      863299160      

              remoteSite  remoteSiteHash run_number  run_numberHash  \
0  TOKYO-LCG2_DATADISK   -2113133296      00280862   1717963924       
1  AGLT2_DATADISK        -1643637004      00284285   1039245728       
2  FMPHI-UNIBA_DATADISK   1301399448      00267073  -360919798        
3  UKI-LT2-RHUL_DATADISK  1896750258      00267162  -2019438915       
4  TOKYO-LCG2_DATADISK   -2113133296      00276181  -1147842459       

    scopeHash   stream_name  stream_nameHash  \
0  1217185487  physics_Main  680461412         
1  1217185487  physics_Main  680461412         
2  1217185487  physics_

In [38]:
df_lookup_reduced.createOrReplaceTempView('lookup_reduced')
spark.sql("SELECT * FROM lookup_reduced WHERE join_key_lookup == 'data15_13TeV:data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00'").toPandas()
#spark.sql("SELECT * FROM lookup_reduced WHERE join_key_lookup IS NULL").toPandas()

SCOPE  \
0    data15_13TeV   
1    data15_13TeV   
2    data15_13TeV   
3    data15_13TeV   
4    data15_13TeV   
5    data15_13TeV   
6    data15_13TeV   
7    data15_13TeV   
8    data15_13TeV   
9    data15_13TeV   
10   data15_13TeV   
11   data15_13TeV   
12   data15_13TeV   
13   data15_13TeV   
14   data15_13TeV   
15   data15_13TeV   
16   data15_13TeV   
17   data15_13TeV   
18   data15_13TeV   
19   data15_13TeV   
20   data15_13TeV   
21   data15_13TeV   
22   data15_13TeV   
23   data15_13TeV   
24   data15_13TeV   
25   data15_13TeV   
26   data15_13TeV   
27   data15_13TeV   
28   data15_13TeV   
29   data15_13TeV   
..            ...   
97   data15_13TeV   
98   data15_13TeV   
99   data15_13TeV   
100  data15_13TeV   
101  data15_13TeV   
102  data15_13TeV   
103  data15_13TeV   
104  data15_13TeV   
105  data15_13TeV   
106  data15_13TeV   
107  data15_13TeV   
108  data15_13TeV   
109  data15_13TeV   
110  data15_13TeV   
111  data15_13TeV   
112  data15_13TeV   
113  data15_13TeV   
114  data15_13TeV   
115  data15_13TeV   
116  data15_13TeV   
117  data15_13TeV   
118  data15_13TeV   
119  data15_13TeV   
120  data15_13TeV   
121  data15_13TeV   
122  data15_13TeV   
123  data15_13TeV   
124  data15_13TeV   
125  data15_13TeV   
126  data15_13TeV   

                                                                                      NAME  \
0    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
1    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
2    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
3    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
4    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
5    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
6    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
7    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
8    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
9    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
10   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
11   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
12   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
13   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
14   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
15   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
16   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
17   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
18   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
19   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
20   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
21   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
22   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
23   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
24   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
25   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
26   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
27   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_t

In [39]:
df_final = df_april.join(df_lookup_reduced, df_april.join_key_april == df_lookup_reduced.join_key_lookup, 'left_outer')

In [40]:
df_final.count()

2855520203

In [41]:
df_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE join_key_lookup IS NULL").limit(10).toPandas()

account  accountHash clientState  clientStateHash  \
0  toyamaza  368790896    DONE       -878527175         
1  toyamaza  368790896    DONE       -878527175         
2  toyamaza  368790896    DONE       -878527175         
3  toyamaza  368790896    DONE       -878527175         
4  toyamaza  368790896    DONE       -878527175         
5  toyamaza  368790896    DONE       -878527175         
6  toyamaza  368790896    DONE       -878527175         
7  toyamaza  368790896    DONE       -878527175         
8  toyamaza  368790896    DONE       -878527175         
9  toyamaza  368790896    DONE       -878527175         

                                                                                                                              dataset  \
0  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
1  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
2  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
3  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
4  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
5  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
6  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
7  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
8  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   
9  mc16_13TeV.364170.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV0_70_CVetoBVeto.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3387   

   datasetHash datasetScope  datasetScopeHash  \
0 -460635832    mc16_13TeV  -1447489496         
1 -460635832    mc16_13TeV  -1447489496         
2 -460635832    mc16_13TeV  -1447489496         
3 -460635832    mc16_13TeV  -1447489496         
4 -460635832    mc16_13TeV  -1447489496         
5 -460635832    mc16_13TeV  -1447489496         
6 -460635832    mc16_13TeV  -1447489496         
7 -460635832    mc16_13TeV  -1447489496         
8 -460635832    mc16_13TeV  -1447489496         
9 -460635832    mc16_13TeV  -1447489496         

                               dataset_version  dataset_versionHash  \
0  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
1  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
2  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
3  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
4  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
5  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
6  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
7  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
8  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             
9  e5340_e5984_s3126_s3136_r10201_r10210_p3387  483295110             

     datatype  datatypeHash eventType  eventTypeHash eventVersion  \
0  DAOD_SUSY5 -506354024     download  59378080       1.14.11       
1  DAOD_SUSY5 -506354024     download  59378080       1.14.11       
2  DAOD_SUSY5 -506354024     download  59378080       1.14.11       
3  DAOD_SUSY5 -506354024     download  59378080       1.14.11       
4  DAOD_SUSY5 -506354024     download  59378080       1.14.11       
5  DAOD_SUSY5 -506354024     download  59378080       1.14.11       
6  DAOD_SUSY5 -506354024     download  59378080       1.14.11  

In [42]:
df_final_dropped = df_final.select('join_key_lookup','CHILD_SCOPE','CHILD_NAME','BYTES')

In [43]:
df_final_dropped.printSchema()

root
 |-- join_key_lookup: string (nullable = true)
 |-- CHILD_SCOPE: string (nullable = true)
 |-- CHILD_NAME: string (nullable = true)
 |-- BYTES: string (nullable = true)



In [44]:
df_final_dropped = df_final_dropped.dropDuplicates()

In [45]:
df_final_dropped.count()

3694965

In [46]:
df_final_dropped.createOrReplaceTempView("final_dropped")
spark.sql("SELECT COUNT(*) FROM final_dropped WHERE join_key_lookup IS NULL").toPandas()
#vermutlich bleibt nur ein eintrag über, weil alle anderen durch das selektive dropduplicates entfernt wurden

count(1)
0  1

In [47]:
df_final_dropped = df_final_dropped.dropna(subset=['join_key_lookup'])

In [48]:
df_final_dropped.count()

3694964

In [49]:
df_final_dropped_json = df_final_dropped.toJSON()
df_final_dropped_json.first()

u'{"join_key_lookup":"data15_13TeV:data15_13TeV.00276262.physics_Main.deriv.DAOD_SUSY11.r9264_p3083_p3372_tid12599464_00","CHILD_SCOPE":"data15_13TeV","CHILD_NAME":"DAOD_SUSY11.12599464._000037.pool.root.1","BYTES":"202471103"}'

In [56]:
df_final_dropped.write.format("json").save("/user/sprenner/lookup_final_small.json")

In [51]:
df_final_dropped_count = df_final_dropped.groupBy('join_key_lookup').count()

In [57]:
df_final_dropped_count.write.format("json").save("/user/sprenner/lookup_final_count.json")

In [53]:
df_final_dropped_count.orderBy('count', ascending=False).toPandas()
#12362 rows mit null drin, sollte eine weniger sein

join_key_lookup  \
0      mc15_13TeV:mc15_13TeV.362455.Sherpa_CT10_Wenu_Pt140_280_BFilter_qsf4.evgen.EVNT.e4542_tid06815745_00                                                                  
1      mc15_13TeV:mc15_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.evgen.EVNT.e5894_tid11041837_00                                                                              
2      mc15_13TeV:mc15_13TeV.364133.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV70_140_BFilter.evgen.EVNT.e5307_tid09300375_00                                                    
3      mc15_13TeV:mc15_13TeV.364136.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV140_280_BFilter.evgen.EVNT.e5307_tid09300387_00                                                   
4      mc15_13TeV:mc15_13TeV.361250.Pythia8B_A14_NNPDF23LO_bbTomu15.evgen.log.e3878_tid11331344_00                                                                           
5      mc16_13TeV:mc16_13TeV.424108.Pythia8B_A14_CTEQ6L1_Jpsimu6.simul.HITS.e5441_s3126_tid11330326_00                                                                       
6      mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid12017482_00                                                   
7      mc15_13TeV:mc15_13TeV.364122.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV140_280_BFilter.evgen.EVNT.e5299_tid09741824_00                                                       
8      mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid12860054_00                                                      
9      mc15_13TeV:mc15_13TeV.345055.PowhegPythia8EvtGen_NNPDF3_AZNLO_ZH125J_MINLO_llbb_VpT.evgen.TXT.e5706_tid12017484_00                                                    
10     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid12458448_00                                                      
11     mc15_13TeV:mc15_13TeV.364108.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV140_280_BFilter.evgen.EVNT.e5271_tid09188509_00                                                     
12     mc15_13TeV:mc15_13TeV.304784.Pythia8EvtGen_A14NNPDF23LO_jetjet_Powerlaw.evgen.EVNT.e4748_tid07422034_00                                                               
13     mc15_13TeV:mc15_13TeV.304784.Pythia8EvtGen_A14NNPDF23LO_jetjet_Powerlaw.evgen.EVNT.e4748_tid07434778_00                                                               
14     mc15_13TeV:mc15_13TeV.364139.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_BFilter.evgen.EVNT.e5313_tid09307168_00                                                   
15     mc15_13TeV:mc15_13TeV.364125.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV280_500_BFilter.evgen.EVNT.e5299_tid09265966_00                                                       
16     mc15_14TeV:mc15_14TeV.427080.Pythia8EvtGen_A14NNPDF23LO_flatpT_Zprime.recon.AOD.e6172_s3142_s3143_r9589_tid11797763_00                                                
17     mc15_13TeV:mc15_13TeV.364130.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV0_70_BFilter.evgen.EVNT.e5307_tid09300364_00                                                      
18     mc15_13TeV:mc15_13TeV.364116.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_BFilter.evgen.EVNT.e5299_tid09265075_00                                                          
19     mc15_13TeV:mc15_13TeV.364102.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV0_70_BFilter.evgen.EVNT.e5271_tid09188497_00                                                        
20     mc15_13TeV:mc15_13TeV.363155.MGPy8EG_N30NLO_Zee_Ht140_280_BFilter.evgen.EVNT.e4866_tid11342168_00                                                                     
21     mc15_13TeV:mc15_13TeV.364119.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV70_140_BFilter.evgen.EVNT.e5299_tid12461100_00                                                        
22     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.recon.AOD.e6337_e5984_s3126_r10201_tid13122597_00                            

In [54]:
df_final_dropped.createOrReplaceTempView("dropped")
df_count = spark.sql("SELECT join_key_lookup, COUNT(DISTINCT *) AS count FROM dropped GROUP BY join_key_lookup ORDER BY count DESC")
df_count.toPandas()
##hier war ganz unten als jetzte die Gruppe "None" mit count 0

join_key_lookup  \
0      mc15_13TeV:mc15_13TeV.362455.Sherpa_CT10_Wenu_Pt140_280_BFilter_qsf4.evgen.EVNT.e4542_tid06815745_00                                                                       
1      mc15_13TeV:mc15_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.evgen.EVNT.e5894_tid11041837_00                                                                                   
2      mc15_13TeV:mc15_13TeV.364133.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV70_140_BFilter.evgen.EVNT.e5307_tid09300375_00                                                         
3      mc15_13TeV:mc15_13TeV.364136.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV140_280_BFilter.evgen.EVNT.e5307_tid09300387_00                                                        
4      mc15_13TeV:mc15_13TeV.361250.Pythia8B_A14_NNPDF23LO_bbTomu15.evgen.log.e3878_tid11331344_00                                                                                
5      mc16_13TeV:mc16_13TeV.424108.Pythia8B_A14_CTEQ6L1_Jpsimu6.simul.HITS.e5441_s3126_tid11330326_00                                                                            
6      mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid12017482_00                                                        
7      mc15_13TeV:mc15_13TeV.364122.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV140_280_BFilter.evgen.EVNT.e5299_tid09741824_00                                                            
8      mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid12860054_00                                                           
9      mc15_13TeV:mc15_13TeV.345055.PowhegPythia8EvtGen_NNPDF3_AZNLO_ZH125J_MINLO_llbb_VpT.evgen.TXT.e5706_tid12017484_00                                                         
10     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid12458448_00                                                           
11     mc15_13TeV:mc15_13TeV.364108.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV140_280_BFilter.evgen.EVNT.e5271_tid09188509_00                                                          
12     mc15_13TeV:mc15_13TeV.304784.Pythia8EvtGen_A14NNPDF23LO_jetjet_Powerlaw.evgen.EVNT.e4748_tid07434778_00                                                                    
13     mc15_13TeV:mc15_13TeV.304784.Pythia8EvtGen_A14NNPDF23LO_jetjet_Powerlaw.evgen.EVNT.e4748_tid07422034_00                                                                    
14     mc15_13TeV:mc15_13TeV.364139.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_BFilter.evgen.EVNT.e5313_tid09307168_00                                                        
15     mc15_13TeV:mc15_13TeV.364125.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV280_500_BFilter.evgen.EVNT.e5299_tid09265966_00                                                            
16     mc15_14TeV:mc15_14TeV.427080.Pythia8EvtGen_A14NNPDF23LO_flatpT_Zprime.recon.AOD.e6172_s3142_s3143_r9589_tid11797763_00                                                     
17     mc15_13TeV:mc15_13TeV.364130.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV0_70_BFilter.evgen.EVNT.e5307_tid09300364_00                                                           
18     mc15_13TeV:mc15_13TeV.364116.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_BFilter.evgen.EVNT.e5299_tid09265075_00                                                               
19     mc15_13TeV:mc15_13TeV.364102.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV0_70_BFilter.evgen.EVNT.e5271_tid09188497_00                                                             
20     mc15_13TeV:mc15_13TeV.363155.MGPy8EG_N30NLO_Zee_Ht140_280_BFilter.evgen.EVNT.e4866_tid11342168_00                                                                          
21     mc15_13TeV:mc15_13TeV.364119.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV70_140_BFilter.evgen.EVNT.e5299_tid12461100_00                                                             
22     mc16_13TeV:mc16_13TeV.410470.PhPy8EG

In [55]:
df_final_dropped_count.createOrReplaceTempView("counts")
spark.sql("SELECT * FROM counts WHERE join_key_lookup == 'None' OR join_key_lookup IS NULL").toPandas()
#hier war ein eintrag, "None" mit count 1

Empty DataFrame
Columns: [join_key_lookup, count]
Index: []